In [1]:
import creating_baskets as cb
import numpy as np

In [2]:
#Creating Baskets
market_baskets = cb.CreateBaskets()

In [3]:
#Making sure it works
for item in market_baskets.values():
    print(item)

['mens1000free', 'mens100free', 'mens200free', 'mens500free']
['mens1000free', 'mens100free', 'mens200free', 'mens500free']
['mens1000free', 'mens200free', 'mens400IM', 'mens500free']
['mens1000free', 'mens400IM', 'mens500free', 'mens50Free']
['mens1000free', 'mens200free', 'mens400IM', 'mens500free']
['mens1000free', 'mens200fly', 'mens400IM', 'mens500free']
['mens1000free', 'mens200im', 'mens500free']
['mens1000free', 'mens200fly', 'mens200free', 'mens500free']
['mens1000free', 'mens500free']
['mens1000free', 'mens200free', 'mens400IM', 'mens500free']
['mens1000free', 'mens200im', 'mens500free']
['mens1000free', 'mens400IM', 'mens500free']
['mens1000free', 'mens200free', 'mens200im', 'mens500free']
['mens1000free', 'mens200fly', 'mens200free', 'mens500free']
['mens1000free', 'mens200free', 'mens400IM', 'mens500free']
['mens1000free', 'mens100free', 'mens200free', 'mens500free']
['mens1000free', 'mens200fly', 'mens200free', 'mens500free']
['mens1000free', 'mens100free', 'mens200free',

In [4]:
#Creating a hash table that links the event to an integer
counter = 0
events_hash = {}


for events in market_baskets.values():
    for event in events:
        if event not in events_hash:
            events_hash[event] = counter
            counter += 1

print(events_hash)
# m = number of total events
#Will be used for later
m = len(events_hash)

{'mens1000free': 0, 'mens100free': 1, 'mens200free': 2, 'mens500free': 3, 'mens400IM': 4, 'mens50Free': 5, 'mens200fly': 6, 'mens200im': 7, 'mens200back': 8, 'mens100fly': 9, 'mens50back': 10, 'mens200breast': 11, 'mens100back': 12, 'mens100i': 13, 'mens50breast': 14, 'mens50fly': 15, 'mens100breast': 16, 'womens1000free': 17, 'womens100free': 18, 'womens200free': 19, 'womens500free': 20, 'womens200im': 21, 'womens400IM': 22, 'womens200back': 23, 'womens100fly': 24, 'womens200fly': 25, 'womens200breast': 26, 'womens100back': 27, 'womens50back': 28, 'womens100im': 29, 'womens50fly': 30, 'womens50breast': 31, 'womens50free': 32, 'womens100breast': 33}


In [5]:
#Calculating all event supports
event_supports = np.zeros(m)

for row in market_baskets.items():
    events = row[1]
    for event in events:
        event_supports[events_hash[event]] += 1

event_supports = event_supports/len(market_baskets)

#print(events_hash[0])

for i in range(len(event_supports)):
    print(i, event_supports[i])

0 0.022786458333333332
1 0.11848958333333333
2 0.11328125
3 0.041666666666666664
4 0.030598958333333332
5 0.1484375
6 0.039713541666666664
7 0.08919270833333333
8 0.03515625
9 0.107421875
10 0.12434895833333333
11 0.047526041666666664
12 0.10091145833333333
13 0.06380208333333333
14 0.10872395833333333
15 0.14778645833333334
16 0.09244791666666667
17 0.017578125
18 0.11848958333333333
19 0.09309895833333333
20 0.02734375
21 0.08919270833333333
22 0.044270833333333336
23 0.041015625
24 0.09049479166666667
25 0.024088541666666668
26 0.048177083333333336
27 0.09375
28 0.109375
29 0.09895833333333333
30 0.11979166666666667
31 0.10091145833333333
32 0.126953125
33 0.08984375


In [6]:
#Reverse event hash that will be used later, links the integer back to the event name
reverse_events_hash = {}
for i in range(m):
    for key, val in events_hash.items():
        if val == i:
            reverse_events_hash[i] = key

print(reverse_events_hash)

{0: 'mens1000free', 1: 'mens100free', 2: 'mens200free', 3: 'mens500free', 4: 'mens400IM', 5: 'mens50Free', 6: 'mens200fly', 7: 'mens200im', 8: 'mens200back', 9: 'mens100fly', 10: 'mens50back', 11: 'mens200breast', 12: 'mens100back', 13: 'mens100i', 14: 'mens50breast', 15: 'mens50fly', 16: 'mens100breast', 17: 'womens1000free', 18: 'womens100free', 19: 'womens200free', 20: 'womens500free', 21: 'womens200im', 22: 'womens400IM', 23: 'womens200back', 24: 'womens100fly', 25: 'womens200fly', 26: 'womens200breast', 27: 'womens100back', 28: 'womens50back', 29: 'womens100im', 30: 'womens50fly', 31: 'womens50breast', 32: 'womens50free', 33: 'womens100breast'}


In [7]:
#Calculating pairs
pairs = {}
#Setting all possible pairs to 0 first, as this will aid us in computing the ideal meet schedule with distances.
#Upper Triangular
for i in range(m):
    for j in range(i+1, m):
        pairs[i,j] = 0

#Looking through all baskets and counting pairs
for basket in market_baskets.values():
    for i in range(len(basket)):
        for j in range(i + 1, len(basket)):
            e1 = events_hash[basket[i]]
            e2 = events_hash[basket[j]]

            if (e1 < e2):
                pairs[e1,e2] += 1
            else:
                pairs[e2,e1] += 1
#print(pairs)

#Ensuring that we have an upper triangular matrix
test_matrix = np.zeros((m,m))
for key, value in pairs.items():
    test_matrix[key[0], key[1]] = value
print(test_matrix)

[[ 0.  5. 22. ...  0.  0.  0.]
 [ 0.  0. 77. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0. 25. 98.]
 [ 0.  0.  0. ...  0.  0. 14.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [8]:
#little test
max_pair = max(pairs, key = pairs.get)
first = max_pair[0]
second = max_pair[1]
print(reverse_events_hash[first], reverse_events_hash[second])

mens100free mens50Free


In [13]:
# Getting most popular pairs
sorted_items = sorted(pairs.items(), key=lambda x: x[1], reverse=True)

# Take top n
top_n = sorted_items[:10]

for k, v in top_n:
    print(f"{reverse_events_hash[k[0]], reverse_events_hash[k[1]]}: {v}")

('mens100free', 'mens50Free'): 119
('mens50back', 'mens100back'): 109
('womens100free', 'womens50free'): 109
('mens50breast', 'mens100breast'): 102
('womens100back', 'womens50back'): 98
('womens50breast', 'womens100breast'): 98
('mens100fly', 'mens50fly'): 97
('mens50Free', 'mens50fly'): 96
('womens50fly', 'womens50free'): 81
('mens100free', 'mens200free'): 77


In [16]:
bottom_n = sorted_items[:-10]
for k, v in bottom_n:
    print(f"{reverse_events_hash[k[0]], reverse_events_hash[k[1]]}: {v}")

('mens100free', 'mens50Free'): 119
('mens50back', 'mens100back'): 109
('womens100free', 'womens50free'): 109
('mens50breast', 'mens100breast'): 102
('womens100back', 'womens50back'): 98
('womens50breast', 'womens100breast'): 98
('mens100fly', 'mens50fly'): 97
('mens50Free', 'mens50fly'): 96
('womens50fly', 'womens50free'): 81
('mens100free', 'mens200free'): 77
('womens100fly', 'womens50fly'): 77
('womens100free', 'womens200free'): 75
('mens100free', 'mens50fly'): 64
('mens200breast', 'mens100breast'): 59
('womens200breast', 'womens100breast'): 58
('womens50back', 'womens50free'): 55
('womens100im', 'womens50fly'): 53
('womens100free', 'womens50fly'): 51
('mens200breast', 'mens50breast'): 49
('mens50back', 'mens50fly'): 48
('womens100im', 'womens100breast'): 48
('mens50Free', 'mens50back'): 46
('womens200im', 'womens100im'): 46
('mens50Free', 'mens100fly'): 43
('womens200back', 'womens100back'): 43
('womens200breast', 'womens50breast'): 43
('womens50back', 'womens50fly'): 43
('mens200fr

In [22]:
print(events_hash["mens1000free"])
print(events_hash["mens500free"])
print(events_hash["mens200free"])
print(events_hash["mens100free"])
print(events_hash["mens50Free"])
print(events_hash["mens100breast"])
print(events_hash["mens200breast"])
print(events_hash["mens50Free"])

0
3
2
1
5
16
11
5


In [ ]:
test_array = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]

#dist(i,j) = support(i,j) * 1/|pos(i) - pos(j)|
def dist(x1,x2, pos1, pos2): 
    if x1 < x2:
        return 1/np.abs(pos1 - pos2) * (pairs[x1,x2])
    else:
        return 1/np.abs(pos1 - pos2) * (pairs[x2,x1])

def calc_score(lineup):
    score = 0
    for i in range(len(lineup)):
        for j in range(i+1, len(lineup)):
            score += dist(lineup[i], lineup[j], i, j)
    return score

print(calc_score(test_array))

1750.2607004107006


In [25]:
print(reverse_events_hash)

{0: 'mens1000free', 1: 'mens100free', 2: 'mens200free', 3: 'mens500free', 4: 'mens400IM', 5: 'mens50Free', 6: 'mens200fly', 7: 'mens200im', 8: 'mens200back', 9: 'mens100fly', 10: 'mens50back', 11: 'mens200breast', 12: 'mens100back', 13: 'mens100i', 14: 'mens50breast', 15: 'mens50fly', 16: 'mens100breast', 17: 'womens1000free', 18: 'womens100free', 19: 'womens200free', 20: 'womens500free', 21: 'womens200im', 22: 'womens400IM', 23: 'womens200back', 24: 'womens100fly', 25: 'womens200fly', 26: 'womens200breast', 27: 'womens100back', 28: 'womens50back', 29: 'womens100im', 30: 'womens50fly', 31: 'womens50breast', 32: 'womens50free', 33: 'womens100breast'}


In [30]:
#CCS meet schedule (must just one day)
ccs_meet_schedule = [22, 4, 23, 8, 25, 6, 26, 11, 30, 15,21 , 7, 32, 5, 16, 33, 19, 2, 27, 12, 17, 0, 28, 10, 24, 9, 31, 14, 18, 1, 29, 13, 20, 3]
print(calc_score(ccs_meet_schedule))

634.7363903038826


In [32]:
max_score = 1000000
best_schedule = []
for i in range(100000):
    random_schedule = np.random.choice(test_array, size = 34, replace = False)
    score = calc_score(random_schedule)
    if (score < max_score):
        max_score = score
        best_schedule = random_schedule

print("max score of:", max_score)
print(best_schedule)

max score of: 633.9016535000725
[21  7 25 15 19 11 12 26  1  4  0 30 16  9 31 28  5 24 17 23 29 10  6 32
  8 33  3 14 27 20 13  2 22 18]
